## Julia code for simulation

In [1]:
## necessary packages

using Distributions
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using Random
using SpecialFunctions # Matern functions
using MLBase         # cross-validation 
using Convex         # compute stacking weights
using MosekTools     # compute stacking weights

In [2]:
include("./utils.jl")

sp_stacking_K_fold_V2 (generic function with 1 method)

In [3]:
# Set the parameters of the simulated data #
p = 2;      # No. covariates
β = [1.0 2.0]; #regression coeff
σ2 = 1.0; ϕ = 7.0; ν = 1.0; τ2 = 1.0; # hyperparmeters in matern

In [4]:
## Generate simulation data ##
Random.seed!(1);
N = 300;                     # No. all positions
N_ho = 100;                  # No. held out positions
ind_mod = 1:(N - N_ho);      # index of training observations
coords = rand(2, N);         # random location over unit square (2 by N)
X = vcat(fill(1.0, (1, N)), rand(Normal(), (1, N)));          # design matrix (p by N)
D = pairwise(Euclidean(), coords, dims = 2);                  # distance matrix
Cov = Symmetric(Maternlu.(UpperTriangular(D), 
        ν = ν, ϕ = ϕ, σ2 = σ2))                               # covariance matrix
z = rand(MvNormal(Cov), 1);                                   # latent process
y = (β * X)[1,:] + z[:,1] + sqrt(τ2) * rand(Normal(), N);     # response

In [5]:
using BenchmarkTools

In [6]:
## candidate values of hyperparameters for stacking ##
deltasq_grid = [0.1, 0.5, 1, 2];
phi_grid = [3, 9, 15, 21];
nu_grid = [0.5, 1, 1.5, 1.75];

In [7]:
## priors parameters ##
Priors = Dict("μβ" => fill(0.0, p), "inv_V_β" => Diagonal(ones(p) * 0.25), 
    "aσ" => 2.0, "bσ" => 2.0);

In [8]:
label = "LSE"; #stacking of means
K_fold = 10;

In [9]:
label = "LP";  #stacking of predictive densities
#J = 300;       # sample size for computing posterior expectation
K_fold = 10;

In [10]:
out = sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 432, label = label, J = 300);
[out[:grid_all][(out[:w].>0.01), :] out[:w][(out[:w].>0.01)]]

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:02



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



5×4 Matrix{Float64}:
 15.0  0.5  0.1  0.0677227
 15.0  1.0  0.5  0.314192
 21.0  1.0  0.5  0.238521
  9.0  0.5  2.0  0.110361
 15.0  0.5  2.0  0.265437

In [12]:
out = sp_stacking_K_fold_V2(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 432, label = label);
[out[:grid_all][(out[:w].>0.01), :] out[:w][(out[:w].>0.01)]]

5×4 Matrix{Float64}:
 15.0  0.5  0.1  0.0677227
 15.0  1.0  0.5  0.314192
 21.0  1.0  0.5  0.238521
  9.0  0.5  2.0  0.110361
 15.0  0.5  2.0  0.265437

In [13]:
@benchmark sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = label)

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01


BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.025 s …   2.114 s  ┊ GC (min … max): 0.67% … 0.78%
 Time  (median):     2.043 s              ┊ GC (median):    0.67%
 Time  (mean ± σ):   2.061 s ± 46.732 ms  ┊ GC (mean ± σ):  0.69% ± 0.09%

  █          █                                            █  
  █▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.03 s         Histogram: frequency by time        2.11 s <

 Memory estimate: 397.51 MiB, allocs estimate: 7520620.

In [14]:
@benchmark sp_stacking_K_fold_V2(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = label)

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  573.106 ms … 701.179 ms  ┊ GC (min … max): 2.22% … 2.12%
 Time  (median):     594.874 ms               ┊ GC (median):    2.05%
 Time  (mean ± σ):   599.801 ms ±  39.375 ms  ┊ GC (mean ± σ):  2.10% ± 0.21%

  ▁▁ █      ▁▁█                                               ▁  
  ██▁█▁▁▁▁▁▁███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  573 ms           Histogram: frequency by time          701 ms <

 Memory estimate: 526.51 MiB, allocs estimate: 1125736.

In [15]:
@benchmark sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = "LSE")

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01


BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.948 s …   2.058 s  ┊ GC (min … max): 0.50% … 0.66%
 Time  (median):     1.949 s              ┊ GC (median):    0.50%
 Time  (mean ± σ):   1.985 s ± 63.181 ms  ┊ GC (mean ± σ):  0.55% ± 0.10%

  █                                                       ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.95 s         Histogram: frequency by time        2.06 s <

 Memory estimate: 408.69 MiB, allocs estimate: 7160943.

In [16]:
@benchmark sp_stacking_K_fold_V2(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = "LSE")

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  527.393 ms … 675.808 ms  ┊ GC (min … max): 1.79% … 2.55%
 Time  (median):     579.731 ms               ┊ GC (median):    2.22%
 Time  (mean ± σ):   577.403 ms ±  47.108 ms  ┊ GC (mean ± σ):  2.68% ± 0.74%

  ▁▁ ▁         ▁       █      ▁    ▁                          ▁  
  ██▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  527 ms           Histogram: frequency by time          676 ms <

 Memory estimate: 537.57 MiB, allocs estimate: 765937.

In [1]:
Threads.nthreads()

1

In [20]:
versioninfo()

Julia Version 1.9.3
Commit bed2cd540a1 (2023-08-24 14:43 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 9 on 8 virtual cores
Environment:
  LD_LIBRARY_PATH = /opt/homebrew/lib:/usr/local/lib:/usr/lib
  JULIA_NUM_THREADS = 8
